In [1]:
import collections
import numpy as np
import tensorflow as tf
 
#-------------------------------数据预处理---------------------------#
 
poetry_file ='D:\python_code\poem\poetry.txt'
 
# 诗集
poetrys = []
with open(poetry_file, "r", encoding='utf-8',) as f:
	for line in f:
		try:
			title, content = line.strip().split(':')
			content = content.replace(' ','')
			if '_' in content or '(' in content or '（' in content or '《' in content or '[' in content:
				continue
			if len(content) < 5 or len(content) > 79:
				continue
			content = '[' + content + ']'
			poetrys.append(content)
		except Exception as e: 
			pass
 
# 按诗的字数排序
poetrys = sorted(poetrys,key=lambda line: len(line))
print('唐诗总数: ', len(poetrys))
 
# 统计每个字出现次数
all_words = []
for poetry in poetrys:
	all_words += [word for word in poetry]
counter = collections.Counter(all_words)
count_pairs = sorted(counter.items(), key=lambda x: -x[1])
words1, _ = zip(*count_pairs)
 
# 取前多少个常用字
words = words1[:len(words1)] + (' ',)
# 每个字映射为一个数字ID
word_num_map = dict(zip(words, range(len(words))))
# 把诗转换为向量形式，参考TensorFlow练习1
to_num = lambda word: word_num_map.get(word, len(words)) #如果get不到则返回len(words)值
poetrys_vector = [ list(map(to_num, poetry)) for poetry in poetrys]
#[[314, 3199, 367, 1556, 26, 179, 680, 0, 3199, 41, 506, 40, 151, 4, 98, 1],
#[339, 3, 133, 31, 302, 653, 512, 0, 37, 148, 294, 25, 54, 833, 3, 1, 965, 1315, 377, 1700, 562, 21, 37, 0, 2, 1253, 21, 36, 264, 877, 809, 1]
#....]
 
# 每次取64首诗进行训练
batch_size = 64
n_chunk = len(poetrys_vector) // batch_size
x_batches = []
y_batches = []
for i in range(n_chunk):
	start_index = i * batch_size
	end_index = start_index + batch_size
 
	batches = poetrys_vector[start_index:end_index]
	length = max(map(len,batches))
	xdata = np.full((batch_size,length), word_num_map[' '], np.int32)
	for row in range(batch_size):
		xdata[row,:len(batches[row])] = batches[row]
	ydata = np.copy(xdata)
	ydata[:,:-1] = xdata[:,1:]
	"""
	xdata             ydata
	[6,2,4,6,9]       [2,4,6,9,9]
	[1,4,2,8,5]       [4,2,8,5,5]
	"""
	x_batches.append(xdata)
	y_batches.append(ydata)

唐诗总数:  34646


In [14]:
print (poetrys[0:3])

['[长宜子孙。]', '[李下无蹊。]', '[罗钳吉网。]']


In [17]:
print (line)

句:暂游大庾，白鹤飞来谁共语？岭畔人家，曾见寒梅几度花。春来春去，人在落花流水处。花满前蹊，藏尽神仙人不知。



In [18]:
title, content = line.strip().split(':')

In [19]:
print (title)
print (content)

句
暂游大庾，白鹤飞来谁共语？岭畔人家，曾见寒梅几度花。春来春去，人在落花流水处。花满前蹊，藏尽神仙人不知。


In [20]:
content = content.replace(' ','')
print (content)

暂游大庾，白鹤飞来谁共语？岭畔人家，曾见寒梅几度花。春来春去，人在落花流水处。花满前蹊，藏尽神仙人不知。


In [23]:
len(content)

52

In [21]:
content1 = '[' + content + ']'
print (content1)

[暂游大庾，白鹤飞来谁共语？岭畔人家，曾见寒梅几度花。春来春去，人在落花流水处。花满前蹊，藏尽神仙人不知。]


In [29]:
print (len(count_pairs))
print (count_pairs[0:10])

6109
[('，', 123098), ('。', 122382), ('[', 34646), (']', 34646), ('不', 13507), ('人', 12003), ('山', 10162), ('风', 9675), ('日', 8882), ('无', 8576)]


In [30]:
print (len(words1))
print (words1[0:10])

6109
('，', '。', '[', ']', '不', '人', '山', '风', '日', '无')


In [31]:
print (len(words))
print (words[0:10])

6110
('，', '。', '[', ']', '不', '人', '山', '风', '日', '无')


In [33]:
print (word_num_map)

{'苑': 617, '讲': 1515, '璇': 3384, '銮': 1763, '害': 2401, '活': 2085, '刻': 1655, '压': 1167, '夭': 2227, '火': 361, '潋': 2731, '挨': 4637, '蓟': 2402, '鹏': 2033, '湜': 4111, '湫': 3447, '挫': 3511, '弓': 1205, '槃': 3448, '坏': 1894, '隈': 2048, '霓': 1720, '营': 777, '此': 39, '蒺': 5279, '蕴': 3019, '飓': 4887, '精': 813, '蟾': 1644, '闤': 5330, '讼': 2519, '透': 1516, '羹': 2601, '蜚': 4888, '缫': 4242, '刑': 2727, '俙': 5331, '惊': 259, '婑': 5997, '铺': 1415, '墟': 2429, '飚': 4112, '齿': 1437, '螺': 2987, '沉': 725, '澥': 4298, '榸': 5332, '嚘': 5333, '貍': 5334, '查': 3064, '冒': 2198, '闵': 4639, '烹': 2057, '讽': 3329, '灭': 933, '佞': 2932, '伽': 3582, '辋': 3845, '绍': 3982, '澼': 4890, '茝': 4891, '越': 531, '倍': 1450, '引': 519, '嚏': 5335, '家': 62, '隩': 5606, '枝': 178, '亲': 377, '踢': 5336, '悢': 4892, '幽': 240, '每': 590, '骏': 2138, '痴': 2369, '鹖': 4113, '郄': 3054, '琅': 2344, '濡': 2951, '殑': 5192, '潮': 593, '哀': 923, '八': 632, '毁': 3164, '胛': 4640, '麴': 5338, '酝': 2520, '逋': 2678, '唼': 3983, '洄': 3127, '仿': 2101, '惺': 3984, '父': 13

In [2]:
print (word_num_map.get)

<built-in method get of dict object at 0x0000022F821075C8>


In [36]:
print (xdata[0,:])
print (ydata[0,:])

[   2  165   14   78   42  212    0 2112  321   19  156   85    1  390  128
 1052  428  355    0  100   35    7   53   20    1  438   75 1286  141  786
    0  532 2666 1163   25 1586    1  392  947  229  469   77    0  122   81
   51  624  987 1100    1 1123   55 1011  356 1778    0  641  323  722 1000
 2720    1  138   40   65    6  864    0  209  724  341  857  978    1    3
 6109 6109 6109 6109]
[ 165   14   78   42  212    0 2112  321   19  156   85    1  390  128 1052
  428  355    0  100   35    7   53   20    1  438   75 1286  141  786    0
  532 2666 1163   25 1586    1  392  947  229  469   77    0  122   81   51
  624  987 1100    1 1123   55 1011  356 1778    0  641  323  722 1000 2720
    1  138   40   65    6  864    0  209  724  341  857  978    1    3 6109
 6109 6109 6109 6109]


In [37]:
len(words)

6110

In [2]:

 
 
#---------------------------------------RNN--------------------------------------#
 
input_data = tf.placeholder(tf.int32, [batch_size, None])
output_targets = tf.placeholder(tf.int32, [batch_size, None])



# 定义RNN
def neural_network(model='lstm', rnn_size=128, num_layers=2):
	if model == 'rnn':
		cell_fun = tf.contrib.rnn.BasicRNNCell
	elif model == 'gru':
		cell_fun = tf.contrib.rnn.GRUCell
	elif model == 'lstm':
		cell_fun = tf.contrib.rnn.BasicLSTMCell
        
	def lstm_cell():
		return tf.contrib.rnn.BasicLSTMCell(128, state_is_tuple=True)
	attn_cell = lstm_cell

	cell = tf.contrib.rnn.MultiRNNCell([attn_cell() for _ in range(num_layers)], state_is_tuple=True) # 多层lstm cell 堆叠起来
        
 
	initial_state = cell.zero_state(batch_size, tf.float32)
 
	with tf.variable_scope('rnnlm'):
		softmax_w = tf.get_variable("softmax_w", [rnn_size, len(words)+1])
		softmax_b = tf.get_variable("softmax_b", [len(words)+1])
        
        
		with tf.device("/cpu:0"):
			embedding = tf.get_variable("embedding", [len(words)+1, rnn_size])
			inputs = tf.nn.embedding_lookup(embedding, input_data)
 
	outputs, last_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, scope='rnnlm')
	output = tf.reshape(outputs,[-1, rnn_size])
 

	logits = tf.matmul(output, softmax_w) + softmax_b
	probs = tf.nn.softmax(logits)
	return logits, last_state, probs, cell, initial_state
#训练
def train_neural_network():
	logits, last_state, _, _, _ = neural_network()
	targets = tf.reshape(output_targets, [-1])  #变为一维
	loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [targets], [tf.ones_like(targets, dtype=tf.float32)], len(words))
	cost = tf.reduce_mean(loss)
	learning_rate = tf.Variable(0.0, trainable=False)
	tvars = tf.trainable_variables()
	grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), 5)
	optimizer = tf.train.AdamOptimizer(learning_rate)
	train_op = optimizer.apply_gradients(zip(grads, tvars))
 
	with tf.Session() as sess:
		sess.run(tf.initialize_all_variables())
 
		saver = tf.train.Saver(tf.all_variables())
 
		for epoch in range(2):
			sess.run(tf.assign(learning_rate, 0.002 * (0.97 ** epoch)))
			n = 0
			for batche in range(n_chunk):
				train_loss, _ , _ = sess.run([cost, last_state, train_op], feed_dict={input_data: x_batches[n], output_targets: y_batches[n]})
				n += 1
				print(epoch, batche, train_loss)
			if epoch % 2 == 0:
				saver.save(sess, 'D:\python_code\poem\poetry.module', global_step=epoch)
 


In [4]:
train_neural_network()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Please use tf.global_variables instead.
0 0 8.71528
0 1 8.71238
0 2 8.70942
0 3 8.69673
0 4 8.68583
0 5 8.62225
0 6 8.5389
0 7 8.29788
0 8 8.06195
0 9 7.82401
0 10 7.55128
0 11 7.24166
0 12 7.02063
0 13 6.81683
0 14 6.62607
0 15 6.58499
0 16 6.46136
0 17 6.38633
0 18 6.29096
0 19 6.24145
0 20 6.121
0 21 6.06033
0 22 6.43721
0 23 6.27691
0 24 6.22626
0 25 6.33746
0 26 6.1935
0 27 6.25122
0 28 6.42659
0 29 6.22827
0 30 6.19656
0 31 6.27708
0 32 6.31842
0 33 6.17292
0 34 6.44073
0 35 6.27517
0 36 6.21189
0 37 6.13297
0 38 6.27875
0 39 6.19655
0 40 6.02904
0 41 6.41255
0 42 6.76317
0 43 6.95116
0 44 6.29256
0 45 6.99127
0 46 7.625
0 47 6.53863
0 48 6.67505
0 49 6.59703
0 50 6.57309
0 51 6.36075
0 52 6.22419
0 53 6.39544
0 54 6.33125
0 55 6.4275
0 56 6.308
0 57 6.27516
0 58 6.195
0 59 6.38059
0 60 6.22021
0 61 6.31642
0 62 6.25315
0 63 6.32084
0 64 6.31028
0 65 6.35885
0 66 6.28689
0 67 6.26

In [40]:
a=np.array([[[1,2,3],[2,3,4]],[[3,4,5],[4,5,6]]])

In [41]:
a.shape

(2, 2, 3)

In [42]:
a[0,:,:]

array([[1, 2, 3],
       [2, 3, 4]])

In [43]:
a[:,0,:]

array([[1, 2, 3],
       [3, 4, 5]])

In [44]:
a[:,:,0]

array([[1, 2],
       [3, 4]])

In [3]:
#-------------------------------生成古诗---------------------------------#  
# 使用训练完成的模型  
   
def gen_poetry():  
    def to_word(weights):  
        t = np.cumsum(weights)  
        s = np.sum(weights)  
        sample = int(np.searchsorted(t, np.random.rand(1)*s))  
        return words[sample]  
   
    _, last_state, probs, cell, initial_state = neural_network()  
   
    with tf.Session() as sess:  
        sess.run(tf.global_variables_initializer())  
   
        saver = tf.train.Saver(tf.all_variables())  
        saver.restore(sess, 'D:\python_code\poem\poetry.module-0')  
   
        state_ = sess.run(cell.zero_state(1, tf.float32))  
   
        x = np.array([list(map(word_num_map.get, '['))])  
        print (x)
        
        [probs_, state_] = sess.run([probs, last_state], feed_dict={input_data: x, initial_state: state_})  
        word = to_word(probs_)  
        #word = words[np.argmax(probs_)]  
        poem = ''  
        while word != ']':  
            poem += word  
            x = np.zeros((1,1))  
            x[0,0] = word_num_map[word]  
            [probs_, state_] = sess.run([probs, last_state], feed_dict={input_data: x, initial_state: state_})  
            word = to_word(probs_)  
            #word = words[np.argmax(probs_)]  
        return poem  
   
print(gen_poetry())

Instructions for updating:
Please use tf.global_variables instead.
INFO:tensorflow:Restoring parameters from D:\python_code\poem\poetry.module-0
[[2]]


ValueError: Cannot feed value of shape (1, 128) for Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0', which has shape '(64, 128)'

In [3]:
def gen_poetry_with_head(head):  
    def to_word(weights):  
        t = np.cumsum(weights)  
        s = np.sum(weights)  
        sample = int(np.searchsorted(t, np.random.rand(1)*s))  
        return words[sample]  
   
    _, last_state, probs, cell, initial_state = neural_network()  
   
    with tf.Session() as sess:  
        sess.run(tf.initialize_all_variables())  
   
        saver = tf.train.Saver(tf.all_variables())  
        saver.restore(sess, 'D:\python_code\poem\poetry.module-0')  
   
        state_ = sess.run(cell.zero_state(1, tf.float32))  
        poem = ''  
        i = 0  
        for word in head:  
            while word != '，' and word != '。':  
                poem += word  
                x = np.array([list(map(word_num_map.get, word))])  
                [probs_, state_] = sess.run([probs, last_state], feed_dict={input_data: x, initial_state: state_})  
                word = to_word(probs_)  
                time.sleep(1)  
            if i % 2 == 0:  
                poem += '，'  
            else:  
                poem += '。'  
            i += 1  
        return poem  
   
print(gen_poetry_with_head('一二三四'))  

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Please use tf.global_variables instead.
INFO:tensorflow:Restoring parameters from D:\python_code\poem\poetry.module-0


ValueError: Cannot feed value of shape (1, 1) for Tensor 'Placeholder:0', which has shape '(64, ?)'

In [14]:
import numpy as np
a=np.array([[1,2,3],[2,3,4]])
t = np.cumsum(a)
print (t)

[ 1  3  6  8 11 15]


In [15]:
np.random.rand(1)

array([ 0.43481658])

In [16]:
s = np.sum(t)
print (t)

[ 1  3  6  8 11 15]


In [17]:
a=np.random.rand(1)*s
print (a)

[ 8.43275441]


In [18]:
np.searchsorted(t,a)

array([4], dtype=int64)

In [ ]:
#生成藏头诗
def gen_poetry_with_head(head):  
    def to_word(weights):  
        t = np.cumsum(weights)  
        s = np.sum(weights)  
        sample = int(np.searchsorted(t, np.random.rand(1)*s))  
        return words[sample]  
   
    _, last_state, probs, cell, initial_state = neural_network()  
   
    with tf.Session() as sess:  
        sess.run(tf.global_variables_initializer())  
   
#         saver = tf.train.Saver(tf.all_variables()) 
        saver = tf.train.Saver()
        saver.restore(sess, 'D:\python_code\poem\poetry.module-49')  
        print ('Read finished')
   
        state_ = sess.run(cell.zero_state(1, tf.float32))  
        poem = ''  
        i = 0  
        for word in head:  
            while word != '，' and word != '。':  
                poem += word  
                x = np.array([list(map(word_num_map.get, word))])  
                [probs_, state_] = sess.run([probs, last_state], feed_dict={input_data: x, initial_state: state_})  
                word = to_word(probs_)  
                time.sleep(1)  
            if i % 2 == 0:  
                poem += '，'  
            else:  
                poem += '。'  
            i += 1  
        return poem  
   
print(gen_poetry_with_head('一二三四'))  

INFO:tensorflow:Restoring parameters from D:\python_code\poem\poetry.module-49
